In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys

sys.path.append("../")
import copy

import logging
from src.utils import logging_utils
from src import functional
from src.models import ModelandTokenizer
from src.dataset import load_dataset, load_relation, fill_template


logger = logging.getLogger(__name__)

logging.basicConfig(
    level=logging.INFO,
    format=logging_utils.DEFAULT_FORMAT,
    datefmt=logging_utils.DEFAULT_DATEFMT,
    stream=sys.stdout,
)

In [3]:
from src.models import ModelandTokenizer

model_name = "meta-llama/Llama-2-7b-hf"
mt = ModelandTokenizer(model_path=model_name)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

2023-12-13 17:10:35 src.models INFO     loaded model <meta-llama/Llama-2-7b-hf> | size: 12980.516 MB


In [4]:
relation = load_relation(
    relation_file = "head_of_government.json",
    num_icl = 0,                          # initialize with zero-shot
    default_path="../data",
    batch_size=500
)

all_samples = copy.deepcopy(relation.samples)

# filter zero-shot model knowledge
relation = functional.filter_samples_by_model_knowledge(
    mt = mt,
    relation = relation,
)

relation.properties["num_icl"] = 5
relation.select_icl_examples(num_icl=5)

relation.samples = all_samples

# filter model knowledge with `num_icl` shots
relation = functional.filter_samples_by_model_knowledge(
    mt = mt,
    relation = relation,
)

2023-12-13 17:10:35 src.dataset INFO     initialized relation -> "head of government" with 500 samples
2023-12-13 17:10:49 src.functional INFO     filtered relation "head of government" to 188 samples (with 0-shots)
2023-12-13 17:11:07 src.functional INFO     filtered relation "head of government" to 372 samples (with 5-shots)


In [5]:
from relations.src.operators import JacobianIclMeanEstimator
import relations.src.functional as relations_functional

# relations_functional.make_prompt = functional.make_prompt
relation.prompt_template

"In <var>, {}'s <role> was named"

In [6]:
relation.few_shot_demonstrations

["In 1994, Serbia's President was named Slobodan Milošević",
 "In 2014, Spain's Prime Minister was named Mariano Rajoy",
 "In 1907, Qatar's Emir was named Sheikh Jassim bin Mohammed Al Thani",
 "In 2006, Australia's Prime Minister was named John Howard",
 "In 1996, Central African Republic's President was named Ange-Félix Patassé"]

In [7]:
estimator = JacobianIclMeanEstimator(
    mt = mt,
    h_layer = 12,
    beta = 5.0
)

lre = estimator(relation)

In [8]:
print(lre.prompt_template)

In 1994, Serbia's President was named Slobodan Milošević
In 2014, Spain's Prime Minister was named Mariano Rajoy
In 1907, Qatar's Emir was named Sheikh Jassim bin Mohammed Al Thani
In 2006, Australia's Prime Minister was named John Howard
In 1996, Central African Republic's President was named Ange-Félix Patassé
In <var>, {}'s <role> was named


In [18]:
correct = 0
wrong = 0
for sample in relation.samples:
    predictions = lre(sample = sample).predictions
    known_flag = functional.is_nontrivial_prefix(
        prediction=predictions[0].token, target=sample.object
    )
    print(f"{sample.subject=} ({sample.placeholders['<var>']}), {sample.object=}, ", end="")
    print(f'predicted="{functional.format_whitespace(predictions[0].token)}", (p={predictions[0].prob:.3f}), known=({functional.get_tick_marker(known_flag)})')
    
    correct += known_flag
    wrong += not known_flag
    
faithfulness = correct/(correct + wrong)

print("------------------------------------------------------------")
print(f"Faithfulness (@1) = {faithfulness}")
print("------------------------------------------------------------")

sample.subject='Afghanistan' (1977), sample.object='Mohammed Daoud Khan', predicted="Ali", (p=0.197), known=(✗)
sample.subject='Turkey' (1994), sample.object='Süleyman Demirel', predicted="I", (p=0.475), known=(✗)
sample.subject='Portugal' (1997), sample.object='António Guterres', predicted="Alfred", (p=0.085), known=(✗)
sample.subject="Côte d'Ivoire" (2015), sample.object='Alassane Ouattara', predicted="K", (p=0.145), known=(✗)
sample.subject='Benin' (2002), sample.object='Mathieu Kérékou', predicted=",", (p=0.148), known=(✗)
sample.subject='Angola' (2015), sample.object='José Eduardo dos Santos', predicted="S", (p=0.183), known=(✗)
sample.subject='Belarus' (2019), sample.object='Alexander Lukashenko', predicted="I", (p=0.222), known=(✗)
sample.subject='Uganda' (2019), sample.object='Yoweri Museveni', predicted="K", (p=0.155), known=(✗)
sample.subject='Luxembourg' (1894), sample.object='Paul Eyschen', predicted="K", (p=0.075), known=(✗)
sample.subject='Germany' (1995), sample.object='